# Initialization

Test notebook for the C-MAPPS benchmark. Test the Tunable Model class architecture

First we import the necessary packages and create the global variables.

In [3]:
import math
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModel import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

from scipy.optimize import differential_evolution

# Create models

Create different types of models to be tested with the Tunable Model class

In [37]:
#Clear the previous tensorflow graph
K.clear_session()

FilterN = 10
FilterL = 10

def RULmodel_SN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(100, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.5))
    model.add(Dense(50, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

def RULmodel_SN_2(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(30, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model


def RULModel_CNN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    print(input_shape)
    
    #Add the layers for the model
    model.add(Conv2D(1, (FilterN, FilterL), input_shape=input_shape, padding='same', kernel_initializer='glorot_normal', activation='relu', name='cl1'))
    model.add(Dropout(0.6))
    model.add(MaxPooling2D(pool_size=5, name='pl1'))
    model.add(Flatten())
    model.add(Dense(20, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model


def RULCNNModel(TW, FeatureN):
    
    input_layer = Input(shape=(TW, FeatureN))
    y = Reshape((TW, FeatureN, 1), input_shape=(TW, FeatureN, ),name = 'Reshape')(input_layer)

    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C1')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C2')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C3')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C4')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C5')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C6')(y)
    
    y = Conv2D(1, 3, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='Clast')(y)  
    
    y = Reshape((TW,14))(y)
    y = Flatten()(y)
    y = Dropout(0.5)(y)
    
    #y = Dense(100, activation='tanh', init='glorot_normal', activity_regularizer=keras.regularizers.l2(0.01),)(y)
    y = Dense(100,activation='tanh', kernel_initializer='glorot_normal', name='fc')(y)
    y = Dense(1)(y)
    
    model = Model(inputs = input_layer, outputs = y, name='RUL_CNN_Model')
    
    return model

# Load the data

In [15]:
#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

windowSize = 25
stride = 2
constRUL = 95


dataFolder = '../CMAPSSData'

#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*windowSize
modelRULSN = RULmodel_SN_2(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel('ModelRUL_SN_1', modelRULSN, selected_features, dataFolder, window_size=windowSize,
                      scaler = min_max_scaler, window_stride=stride, constRul = constRUL)
tModel.loadData(verbose=1, rectify_labels = False)
tModel.printData()

Loading data for dataset 1 with window_size of 25, stride of 2 and constRUL of 95. Cros-Validation ratio 0
Data loaded for dataset 1
Printing shapes

Training data (X, y)
(9141, 350)
(9141, 1)
Testing data (X, y)
(100, 350)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.61403509 -0.01217575 -0.39391206 ... -0.27272727  0.38461538
   0.50126369]
 [-0.24210526 -0.10269984 -0.23156708 ... -0.27272727  0.45299145
   0.55293457]
 [-0.22807018 -0.37480148 -0.14047351 ... -0.27272727  0.35042735
   0.30721707]
 [-0.15087719  0.12652197 -0.52153326 ... -0.45454545  0.36752137
   0.238697  ]
 [-0.40350877  0.05558497 -0.65546787 ... -0.45454545  0.17948718
   0.28166245]]
[[95.]
 [95.]
 [95.]
 [95.]
 [95.]]
Testing data (X, y)
[[-0.41052632 -0.34886183 -0.20270575 ... -0.27272727  0.14529915
   0.29458017]
 [ 0.44561404  0.01270513 -0.20901917 ... -0.63636364  0.14529915
   0.0322943 ]
 [ 0.4877193  -0.02223399  0.02502818 ...  0.09090909  0.35042735
   0.06655434]
 [-0.15087719 

In [12]:
#tModel.printData(printTop=False)
#tModel.windowSize = 20
#tModel.changeDataset(2)
#tModel.loadData(verbose=1, crossValRatio=0.1)
#tModel.printData(printTop=False)
tModel.getModelDescription()
ruldcnn = RULCNNModel(tModel.windowSize, len(selected_features))
ruldcnn.summary()

Description for model: ModelRUL_SN_1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 30)                10530     
_________________________________________________________________
dropout_4 (Dropout)          (None, 30)                0         
_________________________________________________________________
fc2 (Dense)                  (None, 10)                310       
_________________________________________________________________
dropout_5 (Dropout)          (None, 10)                0         
_________________________________________________________________
out (Dense)                  (None, 1)                 11        
Total params: 10,851
Trainable params: 10,851
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output

C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:60: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), name="C1", activation="tanh", padding="same", kernel_initializer="glorot_normal")`
C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:61: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), name="C2", activation="tanh", padding="same", kernel_initializer="glorot_normal")`
C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:62: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), name="C3", activation="tanh", padding="same", kernel_initializer="glorot_normal")`
C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:63: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), name="C4", activation="tanh", padding="same", kernel_initializer="g

# Train the model and test some Tunable Model functionalities

In [16]:
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

#print(tModel.currentModelName)
#tModel.getAllModelsDescription()
#tModel.getModelDescription('RULModel_CNN_1')
#tModel.getModelDescription(tModel.currentModelName)
#tModel.getModelDescription()
#tModel.setCurrentModel('RULmodel_SN_1')
#tModel.getModelDescription()
tModel.epochs = 250
tModel.trainModel(learningRateScheduler=lrate, verbose=1)

Epoch 1/250
9141/9141 [==============================] - 0s 16us/step - loss: 5641.8454 - mean_squared_error: 5641.8454
Epoch 2/250
9141/9141 [==============================] - 0s 8us/step - loss: 4875.8652 - mean_squared_error: 4875.8652
Epoch 3/250
9141/9141 [==============================] - 0s 8us/step - loss: 3668.3489 - mean_squared_error: 3668.3489
Epoch 4/250
9141/9141 [==============================] - 0s 8us/step - loss: 2472.6075 - mean_squared_error: 2472.6075
Epoch 5/250
9141/9141 [==============================] - 0s 8us/step - loss: 1774.2416 - mean_squared_error: 1774.2416: 0s - loss: 1815.1465 - mean_squared_error: 1815.14
Epoch 6/250
9141/9141 [==============================] - 0s 8us/step - loss: 1444.9062 - mean_squared_error: 1444.9062
Epoch 7/250
9141/9141 [==============================] - 0s 9us/step - loss: 1188.1167 - mean_squared_error: 1188.1167
Epoch 8/250
9141/9141 [==============================] - 0s 8us/step - loss: 1001.4960 - mean_squared_error: 1001.

9141/9141 [==============================] - 0s 8us/step - loss: 397.2441 - mean_squared_error: 397.2441
Epoch 70/250
9141/9141 [==============================] - 0s 8us/step - loss: 409.0899 - mean_squared_error: 409.0899
Epoch 71/250
9141/9141 [==============================] - 0s 9us/step - loss: 399.7576 - mean_squared_error: 399.7576
Epoch 72/250
9141/9141 [==============================] - 0s 9us/step - loss: 404.6819 - mean_squared_error: 404.6819
Epoch 73/250
9141/9141 [==============================] - 0s 9us/step - loss: 393.0447 - mean_squared_error: 393.0447
Epoch 74/250
9141/9141 [==============================] - 0s 9us/step - loss: 405.4342 - mean_squared_error: 405.4342
Epoch 75/250
9141/9141 [==============================] - 0s 11us/step - loss: 391.6877 - mean_squared_error: 391.6877
Epoch 76/250
9141/9141 [==============================] - 0s 9us/step - loss: 409.6581 - mean_squared_error: 409.6581
Epoch 77/250
9141/9141 [==============================] - 0s 9us/ste

9141/9141 [==============================] - 0s 8us/step - loss: 386.8683 - mean_squared_error: 386.8683
Epoch 139/250
9141/9141 [==============================] - 0s 9us/step - loss: 377.9265 - mean_squared_error: 377.9265
Epoch 140/250
9141/9141 [==============================] - 0s 8us/step - loss: 386.6231 - mean_squared_error: 386.6231
Epoch 141/250
9141/9141 [==============================] - 0s 8us/step - loss: 386.0409 - mean_squared_error: 386.0409
Epoch 142/250
9141/9141 [==============================] - 0s 8us/step - loss: 375.1592 - mean_squared_error: 375.1592
Epoch 143/250
9141/9141 [==============================] - 0s 8us/step - loss: 386.1270 - mean_squared_error: 386.1270
Epoch 144/250
9141/9141 [==============================] - 0s 9us/step - loss: 373.6916 - mean_squared_error: 373.6916
Epoch 145/250
9141/9141 [==============================] - 0s 9us/step - loss: 395.1188 - mean_squared_error: 395.1188
Epoch 146/250
9141/9141 [==============================] - 0s 

9141/9141 [==============================] - 0s 8us/step - loss: 380.6277 - mean_squared_error: 380.6277
Epoch 207/250
9141/9141 [==============================] - 0s 9us/step - loss: 377.2358 - mean_squared_error: 377.2358
Epoch 208/250
9141/9141 [==============================] - 0s 10us/step - loss: 378.4176 - mean_squared_error: 378.4176
Epoch 209/250
9141/9141 [==============================] - 0s 12us/step - loss: 365.6951 - mean_squared_error: 365.6951
Epoch 210/250
9141/9141 [==============================] - 0s 8us/step - loss: 378.0158 - mean_squared_error: 378.0158
Epoch 211/250
9141/9141 [==============================] - 0s 8us/step - loss: 374.6154 - mean_squared_error: 374.6154
Epoch 212/250
9141/9141 [==============================] - 0s 8us/step - loss: 380.7950 - mean_squared_error: 380.7950
Epoch 213/250
9141/9141 [==============================] - 0s 9us/step - loss: 382.5692 - mean_squared_error: 382.5692
Epoch 214/250
9141/9141 [==============================] - 0

In [17]:
tModel.evaluateModel(["rhs"], round=2)
print("scores")

#print(tModel.y_pred)

cScores = tModel.scores
rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.trainTime

i = 1
for prediction in tModel.y_pred:
    print('Engine {}, RUL {}, Rounded RUL {}, Real RUL {}'.format(i, prediction, 
                                                                    tModel.y_pred_rectified[i-1], 
                                                                    tModel.y_test[i-1]))
    i = i+1

print(cScores)
print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 449us/step
scores
Engine 1, RUL [73.54826], Rounded RUL [73.], Real RUL [112.]
Engine 2, RUL [82.2571], Rounded RUL [82.], Real RUL [98.]
Engine 3, RUL [44.205147], Rounded RUL [44.], Real RUL [69.]
Engine 4, RUL [62.331158], Rounded RUL [62.], Real RUL [82.]
Engine 5, RUL [69.92357], Rounded RUL [69.], Real RUL [91.]
Engine 6, RUL [68.83957], Rounded RUL [68.], Real RUL [93.]
Engine 7, RUL [79.43979], Rounded RUL [79.], Real RUL [91.]
Engine 8, RUL [53.364048], Rounded RUL [53.], Real RUL [95.]
Engine 9, RUL [80.5633], Rounded RUL [80.], Real RUL [111.]
Engine 10, RUL [69.6054], Rounded RUL [69.], Real RUL [96.]
Engine 11, RUL [56.000633], Rounded RUL [56.], Real RUL [97.]
Engine 12, RUL [69.131676], Rounded RUL [69.], Real RUL [124.]
Engine 13, RUL [65.212456], Rounded RUL [65.], Real RUL [95.]
Engine 14, RUL [65.67771], Rounded RUL [65.], Real RUL [107.]
Engine 15, RUL [71.65181], Rounded RUL [71.], Real RUL [83.]
Engine 16, RUL [74.5294

In [ ]:
def nn_optmize_fun(x, selected_features=['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32'], datasetNumber = '1', scaler = None, verbose=0, epochs=250, 
                  saveToFile = None, iterations = 0):
    
    #Clear the previous tensorflow graph
    K.clear_session()
    
    #Extract the tunning variables from the input vector
    #Round the values to the nearest integer since this implementation is for real numbers
    x = x.astype(int)
    windows_size = x[0]
    window_stride = x[1]
    constantRUL = x[2]
    
    if iterations == 0:
        print("Creating model")
    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    
    #Define the model
    nFeatures = len(selected_features)
    shapeSN = nFeatures*windows_size
    modelRULSN = RULmodel_SN_1(shapeSN)
    modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    #load the data using the selected parameters
    tModel = TunableModel(selected_features, scaler = min_max_scaler, window_stride = window_stride, 
                          window_size = windows_size, constRul = constantRUL, datasetNumber = datasetNumber, 
                         epochs=epochs)
    
    tModel.loadData()
    
    #Add the models to the tunableModel object
    tModel.addModel("RUL_SN_1_optmizable", modelRULSN)
    
    if iterations == 0:
        print("Training model")
    #Train the model
    lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
    tModel.trainCurrentModel(learningRateScheduler=lrate, verbose=verbose)
    time = tModel.getModelTimes(tModel.currentModelName)
    
    if iterations == 0:
        print("Training time {}".format(time))
    
    if iterations == 0:
        print("Assesing model performance")
    #Assess the model performance
    tModel.evaluateCurrentModel(["rhs"], crossValidation=True)
    cScores = tModel.getModelScores(tModel.currentModelName)
    rmse = math.sqrt(cScores['score_1'])
    rhs = cScores['rhs']
    #print("The score for this model is: {}".format(rmse))
    
    msgStr = "The model variables are " + str(x) + "\tThe scores are: [RMSE:{:.4f}, RHS:{:.4f}]\n".format(rmse, rhs)
    
    if saveToFile is not None:
        #print(msgStr)
        saveToFile.write(msgStr)
    else:
        print(msgStr)
    
    #Return RMSE as the performance metric to steer the search
    return rmse

In [ ]:
def plotRUL(cycles, rulArray, engineUnit):
    
    plt.clf()
    plt.plot(cycles, rulArray, 'bo-', label='RUL')
    plt.xlabel("Time (Cycle)")
    plt.ylabel("RUL")
    plt.title("Test Engine Unit #{}".format(engineUnit))
    
    plt.show()
    
def plotRUL_Engine(tModel, engineUnit, selectedFeatures, time_window, constRUL, stride, y_test):

    df = tModel.df_test
    df_engine = df[df['Unit Number'] == engineUnit]
    
    X_test2, _, _, _ = CMAPSAuxFunctions.create_windowed_data(df_engine, selected_features, 'test', time_window=time_window, 
                                                     constRUL=constRUL, stride=stride)
    
    X_test2 = tModel.dataScaler.transform(X_test2)
    nnPred = tModel.model.predict(X_test2)
    
    maxCycle = X_test2.shape[0]
    faultCycle = tModel.y_test[engineUnit-1]
    cycles = np.arange(maxCycle)
    rulArray = np.arange(faultCycle, maxCycle+faultCycle)
    rulArray[rulArray > constRUL] = constRUL
    rulArray = np.flipud(rulArray)
    
    plotRUL(cycles, rulArray, engineUnit)
    plt.plot(cycles, nnPred, 'go-', label='NN Pred')
    plt.legend()

In [ ]:
plotRUL_Engine(tModel, engineUnit, selectedFeatures, time_window, stride, constRUL)